In [1]:
# Load the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import keras
from keras import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from matplotlib.image import imread

In [2]:
train_dir = "/kaggle/input/fruit10dataset/train/train"
test_dir  = "/kaggle/input/fruit10dataset/test/test"

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_size = (150,150)
batch_size = 16

# Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

Found 4818 images belonging to 10 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,    
    class_mode='categorical')

Found 1615 images belonging to 10 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same', input_shape = (150, 150, 3)))
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding= 'same', input_shape = (150, 150, 3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(MaxPooling2D((3, 3)))
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer='he_uniform', padding = 'same'))
model.add(MaxPooling2D((3, 3)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation = 'softmax'))

2022-05-18 17:21:46.910798: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 17:21:47.060541: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 17:21:47.061430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 17:21:47.062911: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
from tensorflow.keras.optimizers import SGD

opt = SGD(lr = 0.01, momentum = 0.9)
model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:
history = model.fit(train_generator, batch_size = 64, epochs = 20, validation_data = test_generator, verbose = 1)

2022-05-18 17:21:50.069004: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-05-18 17:21:51.622057: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


302/302 [==============================] - 64s 188ms/step - loss: 1.8504 - accuracy: 0.2931 - val_loss: 1.7565 - val_accuracy: 0.3969
Epoch 2/20
302/302 [==============================] - 36s 119ms/step - loss: 1.0128 - accuracy: 0.5558 - val_loss: 1.2732 - val_accuracy: 0.4669
Epoch 3/20
302/302 [==============================] - 35s 116ms/step - loss: 1.2290 - accuracy: 0.4467 - val_loss: 2.1394 - val_accuracy: 0.4941
Epoch 4/20
302/302 [==============================] - 35s 117ms/step - loss: 0.9222 - accuracy: 0.5654 - val_loss: 2.6369 - val_accuracy: 0.6012
Epoch 5/20
302/302 [==============================] - 35s 116ms/step - loss: 0.7521 - accuracy: 0.6949 - val_loss: 2.0849 - val_accuracy: 0.8074
Epoch 6/20
302/302 [==============================] - 36s 120ms/step - loss: 0.7052 - accuracy: 0.7146 - val_loss: 2.2523 - val_accuracy: 0.8050
Epoch 7/20
302/302 [==============================] - 36s 120ms/step - loss: 0.4929 - accuracy: 0.8022 - val_loss: 3.1500 - val_accuracy: 0.8

In [8]:
score = model.evaluate(test_generator, verbose = 0)
print(score)

[2.004011392593384, 0.20309597253799438]


In [9]:
y_pred = model.predict(test_generator)

In [10]:
from sklearn.metrics import classification_report

preds=model.predict(test_generator)
y_pred = np.argmax(preds,axis=1)
y_actual = test_generator.classes

In [11]:
labels = list(test_generator.class_indices.keys())
print(labels)

['Apple Red 1', 'Banana', 'Blueberry', 'Cherry 1', 'Grape Pink', 'Kiwi', 'Lemon', 'Mango', 'Orange', 'Watermelon']


In [12]:
print(classification_report(y_actual, y_pred, target_names=labels))

              precision    recall  f1-score   support

 Apple Red 1       0.11      1.00      0.20       164
      Banana       0.00      0.00      0.00       166
   Blueberry       0.00      0.00      0.00       154
    Cherry 1       1.00      1.00      1.00       164
  Grape Pink       0.00      0.00      0.00       164
        Kiwi       0.00      0.00      0.00       156
       Lemon       0.00      0.00      0.00       164
       Mango       0.00      0.00      0.00       166
      Orange       0.00      0.00      0.00       160
  Watermelon       0.00      0.00      0.00       157

    accuracy                           0.20      1615
   macro avg       0.11      0.20      0.12      1615
weighted avg       0.11      0.20      0.12      1615



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
